09/04/2020

post scan scrambling the the homotype motifs

In [242]:
# basic packages
import re
import os, glob
import pandas as pd
import numpy as np; np.random.seed(0)
import itertools
from collections import Counter, defaultdict
import time
# Import tools needed for visualization
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt
import pybedtools
from random import shuffle


In [243]:
input_dir = '../data/processed/fig4_modelling/vocab_sum_final/'


In [244]:
save_dir = '../data/processed/fig4_modelling/vocab_sum_final_post_scan/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 0. Setup

## 0A Global Variables

In [245]:
tissue_to_celltype_mapper = {
    'Colon':['Colon'],
    'GDSD6':['GDSD6'],
    'GM12878':['GM12878'],
    'Melanocytes':['Melanocytes'],
    'SCC':['A431-CTRLi','CAL27-CTRLi','SCC13-CTRLi'],
    'MM':['COLO-SCR-DMSO','WM-SCR-DMSO']
}

In [246]:
# %%time
# # import
# data_all = pd.read_csv('/Users/mguo123/Google Drive/1_khavari/omics_project-LD/pan_omics/data/processed/tissue_crms/all_count_comb_overall.csv',index_col=0,header=0)
# data_all = data_all[data_all.tissue.isin(['Colon','GDSD6','GM12878','Melanocytes','A431-CTRLi','CAL27-CTRLi','SCC13-CTRLi','COLO_SCR_DMSO','WM_SCR_DMSO'])]
# data_all = data_all[data_all.iloc[:,2:].sum(axis=1)>1e-1]

# # expression labels
# exp_label = list(np.log10(data_all.exp.values+1e-2))
# labels_all  = np.array(np.array(exp_label)>THRES)


# tissues_label  = data_all.tissue.values#np.array((data_all.exp>THRES).values)
# tissue_num_labels =  data_all.tissue.map(tissues_dict).values
# is_cancer_label = np.array(['cancer' if x in cancer_tissues  else 'normal' for x in tissues_label ])
# tissues_label[tissues_label=='COLO_SCR_DMSO'] = 'COLO-SCR-DMSO'
# tissues_label[tissues_label=='WM_SCR_DMSO'] = 'WM-SCR-DMSO'

# genes_all = data_all.index.values
# gene_to_num_dict = dict(zip(np.unique(genes_all),range(len(np.unique(genes_all)))))
# genes_num_all = np.vectorize(gene_to_num_dict.get)(genes_all)


In [247]:
tss_df = pd.read_csv('../data/external/TSS_annon_hg19.csv',index_col=0).drop_duplicates('TSS')
tss_df

,chr,loc,end,width,strand,TSS
0,chr1,1846266,1846266,1,*,CALML6
1,chr1,231376933,231376933,1,*,C1orf131
2,chr1,241683061,241683061,1,*,FH
3,chr1,120202421,120202421,1,*,PHGDH
4,chr1,204135465,204135465,1,*,REN
...,...,...,...,...,...,...
20255,chrX,140084756,140084756,1,*,SPANXB2
20256,chrX,52683950,52683950,1,*,SSX7
20257,chrX,140096761,140096761,1,*,SPANXB1
20258,chrX,52736239,52736239,1,*,SSX2


In [248]:
# %%time
# tissue_to_foot_df = {}
# foot_tissues = ['Colon','GDSD6','GM12878','Melanocytes','A431-CTRLi','CAL27-CTRLi','SCC13-CTRLi','COLO-SCR-DMSO','WM-SCR-DMSO']
# for tissue in foot_tissues:
#     foot_df = pd.read_csv(os.path.join('../data/interim/annon/promoter_motif_scan/','promoter_'+tissue+'_annon.bed'),sep='\t',header=None)
#     foot_df.columns = ['chr','start','stop','TSS','chr_m','start_m','stop_m','id_trim','score','strand']
#     tissue_to_foot_df[tissue] = foot_df

In [249]:
tf_annon_df = pd.read_csv('../data/external/HOCOMOCOv11_annotation.csv',index_col=0)
tf_annon_df['id_trim'] = tf_annon_df['id'] + '.pwm.trim'
tf_name_to_id_dict = pd.Series(tf_annon_df.id_trim.values, index=tf_annon_df.tf.values).to_dict()
tf_id_to_name_dict = pd.Series(tf_annon_df.tf.values, index=tf_annon_df.id_trim.values).to_dict()
tf_name_to_id_abbr_dict = pd.Series(tf_annon_df.id.values, index=tf_annon_df.tf.values).to_dict()
tf_id_abbr_to_name_dict = pd.Series(tf_annon_df.tf.values, index=tf_annon_df.id.values).to_dict()


In [250]:
MAX_DIST=135


In [251]:

# define constants
LEN_FILLER = 20
NUM_OLIGO_PER_MOTIF = 10
DESIRED_OLIGO_LENGTH = 221#150
MAX_OLIGO_LENGTH = 230
MAX_MOTIF_LENGTH = 63
FORWARD_PCR_PRIMER = 'ACTGGCCGCTTCACTG'
REVERSE_PCR_PRIMER = 'AGATCGGAAGAGCGTCG'
# LINKER_SPACER_v10 = 'CAGTATGCGATGCTCATGATTGTGACTGAAAGTCTATGAGTCCATTGA'
RS_ECORI = 'GAATTC' # 5'-3'
RS_BAMHI = 'GGATCC'
RS_XHOI = 'CTCGAG'
RS_XBAI = 'TCTAGA'
RS_BSPDI = 'ATCGAT'
RS_NHEI = 'GCTAGC'
RS_NCOI = 'CCATGG'

# CCTAGG formed by RS_XHOI and RS_NHEI, can be cut by AvrII

# RS_LIST = [RS_ECORI ,RS_BAMHI , RS_XHOI , RS_XBAI] #original
# RS_LIST = [RS_BSPDI ,RS_BAMHI , RS_XHOI, RS_NHEI]
RS_LIST = [RS_ECORI ,RS_BAMHI , RS_XHOI , RS_NHEI]

RS_XBAI_dam1 = 'GATCTAGA'
RS_XBAI_dam2 = 'TCTAGATC'
RS_XBAI_dam1_pre = '^GA'
RS_XBAI_dam2_suf = 'TC$'
LETTERS=['A', 'C', 'G', 'T']

# 0B helper functions

In [473]:

def check_rs(seq, rs_list = RS_LIST):
    rs_sites = 0
    for rs in rs_list:
        if rs in seq:
            rs_sites+=1
    return rs_sites



def ranges(nums):
    # get consecutive integers in a list
    nums = sorted(set(nums))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s+1 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))

def mixup(word):
    as_list_of_letters = list(word)
    shuffle(as_list_of_letters)
    return ''.join(as_list_of_letters)


def get_vocab_loc(row, motif_len_scr=10):
    # getting vocab locations in the sequence
    vocab1_med = int((row['vocab1_start']+row['vocab1_stop'])/2)
    vocab1_start_shift = vocab1_med - motif_len_scr - row['start_pad']
    vocab1_stop_shift = vocab1_med + motif_len_scr -row['start_pad'] + 1
    
    vocab2_med = int((row['vocab2_start']+row['vocab2_stop'])/2)
    vocab2_start_shift = vocab2_med - motif_len_scr - row['start_pad']
    vocab2_stop_shift = vocab2_med + motif_len_scr -row['start_pad'] + 1
    
    # for overlaps
    if vocab1_stop_shift>= vocab2_start_shift:
#         print(idx)
#         print(row)
        
        # case 1 vocab words are really close, the padding goes into the vocab word
        if (row['vocab1_stop']-row['start_pad'] + 1) >=vocab2_start_shift:
            
            vocab1_stop_shift = row['vocab1_stop']-row['start_pad'] + 1
            vocab2_start_shift = row['vocab1_stop']-row['start_pad'] + 1
#             print(vocab1_start_shift,vocab1_stop_shift, vocab2_start_shift, vocab2_stop_shift)
#             raise
        else: 
            vocab1_stop_shift = vocab1_stop_shift
            vocab2_start_shift = vocab1_stop_shift
#             print(vocab1_start_shift,vocab1_stop_shift, vocab2_start_shift, vocab2_stop_shift)
#             raise

    # for edges:
    vocab1_start_shift = max(0,vocab1_start_shift)
    vocab2_stop_shift = min(len(row['seq']),vocab2_stop_shift)
    
    return pd.Series({'vocab1_start_shift': vocab1_start_shift,
                      'vocab1_stop_shift':vocab1_stop_shift, 
                      'vocab2_start_shift':vocab2_start_shift, 
                      'vocab2_stop_shift':vocab2_stop_shift})


def get_vocab_to_loc_dict(row):
    return {row['vocab1']:[row['vocab1_start_shift'],row['vocab1_stop_shift']],
           row['vocab2']:[row['vocab2_start_shift'],row['vocab2_stop_shift']]}



def scramble_off_target(row,motif_len_scr=10, verbose=False):
    # get places that we can't scramble
    # don't change anything if no homotypic sequences
    if row['off_target_name']=='':
        return row['seq']

    # get vocab lengths
    vocab_word_to_loc = get_vocab_to_loc_dict(row)
    if verbose:
        print('vocab_word_to_loc', vocab_word_to_loc)
    # start mask defaults
    mask = np.zeros(len(row['seq'][0]),dtype=int)

    for motif,pos_str in zip(row['off_target_motif'].split('::'),row['off_target_pos_str'].split('::')):
        pos_arr = [int(x) for x in pos_str.split('|')]
#         print(motif, pos_arr, vocab_word_to_loc[motif])

        v_start, v_stop = vocab_word_to_loc[motif]
        v_len = v_stop - v_start
        for pos in pos_arr:
#             ### OPTION 1 (do +/- 10 bp)
#             pos_med = int(pos+v_len/2)
#             pos_start = pos_med - motif_len_scr
#             pos_stop = pos_med + motif_len_scr + 1
            
            ### option 2 (just scr motif area)
            pos_start = max(0,pos)
            pos_stop = min(len(row['seq'][0]),pos_start + v_len)
            
            mask[pos_start:pos_stop] = 1

    # only considering masking non vocab areas
    mask[row['vocab1_start_shift']:row['vocab1_stop_shift']] = 2
    mask[row['vocab2_start_shift']:row['vocab2_stop_shift']] = 2
    if verbose:
        print('mask',mask)
    mask_indices = np.where(mask==1)[0]
    
    scr_ranges = ranges(mask_indices)
    if verbose:
        print('scr_ranges',scr_ranges)
        
    new_seqs = row['seq'].copy()
    if len(new_seqs)!=4:
        raise ValueError('should have 4 sequences')
#     print(len(new_seqs))
    if len(scr_ranges)>0:
        # iterate through scr ranges
        for (scr_start, scr_stop) in scr_ranges:
            # always take the first sequence (the A_B_seq config)
            seq_scr_ori = new_seqs[0][scr_start:scr_stop]
            seq_scr = mixup(seq_scr_ori)
            for seq_idx in range(len(new_seqs)):
                new_seqs[seq_idx] = new_seqs[seq_idx][:scr_start] + seq_scr + new_seqs[seq_idx][scr_stop:]
                
    # check if the first sequence length is the same
    if len(new_seqs[0])!=len(row['seq'][0]):
        print(new_seqs[0])
        print(row['seq'][0])
        raise ValueError('seq lengths are not the same post scrambling')

    return new_seqs


def off_target_pos_str_find_num(pos_str):
    pos_arr_per_vocab_word = pos_str.split('::')
    if len(pos_arr_per_vocab_word)>0:
        arr = []
        for pos in pos_arr_per_vocab_word:
            for x in pos.split(':'):
                try:
                    arr.append(int(x))
                except:
                    continue
        return len(set(arr))
    return 0

#     vocab1_seq = row['sequence'][vocab1_start_shift:vocab1_stop_shift]
#     vocab2_seq = row['sequence'][vocab2_start_shift:vocab2_stop_shift]

#     vocab1_seq_scr = mixup(vocab1_seq)
#     vocab2_seq_scr = mixup(vocab2_seq)

#     A_B_seq = row['sequence']
#     scrA_B_seq = row['sequence'][:vocab1_start_shift] + vocab1_seq_scr +  row['sequence'][vocab1_stop_shift:]
#     A_scrB_seq = row['sequence'][:vocab2_start_shift] + vocab2_seq_scr +  row['sequence'][vocab2_stop_shift:]
#     scrA_scrB_seq = row['sequence'][:vocab1_start_shift] + vocab1_seq_scr +  row['sequence'][vocab1_stop_shift:vocab2_start_shift] + vocab2_seq_scr +  row['sequence'][vocab2_stop_shift:]

#     # FOR DEBUG
#     print(A_B_seq)
#     print(scrA_B_seq)
#     print(A_scrB_seq)
#     print(scrA_scrB_seq)

#     return A_B_seq, scrA_B_seq, A_scrB_seq, scrA_scrB_seq

# 1. Read in df for building library (vocab and add in homotypic locations

#### RECALL CASES


cases for the motif in the 
1. scrambling removes motif 
2. there's a dummy motif somewhere else in the genomic instance (????? How would you deal with these “noisy” cases?
    2A. is the dummy motif is present in all 4 cases, (not changed by scrambling)
    2B. the dummy motif is present in only some cases, so probably changed by scrambling the other site?
3. scrambling reintroduces the motif at a (hopefully) lower strength, and sometimes with a shifted start position  ??? Potential to rescramble (but sometimes motifs have too many repetitive nucleotides so this might not work…
    3A. the scrambling introduces the motif at a lower strength
    3B. the scrambling does not lower the score of the motif
4. we don't detect the motif in any config in the genomic instance at a certain p value threshold (this usually isn’t the case, if we raise the p-value threshold enough; but the motif will be detected in a lot more genomic instances). This is sort of the sensitivity vs specificity tradeoff.


todo
for case 2A (homotype motifs) scramble those)

1. map the vocab word locations to motifs and match to case above (get the score and the case) - exact match
2. determine vocab words gained lost between the motif configurations

In [471]:

vocab_instances20_seqs_sel = pd.read_csv(os.path.join(input_dir, 'vocab_instances20_seqs_sel.csv'),index_col=0)
pos_genes_df_rs_filt = pd.read_csv(os.path.join(input_dir, 'pos_genes_position_wseq_rsfilt.csv'),index_col=0)
neg_genes_df_rs_filt = pd.read_csv(os.path.join(input_dir, 'neg_genes_position_wseq_rsfilt.csv'),index_col=0)

In [441]:
vocab_instances20_seqs_sel.columns

Index(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
       'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
       'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type',
       'vocab_tissue_count', 'name', 'check_rs', 'scr_pass_rs', 'seq_config',
       'seq', 'name_config'],
      dtype='object')

In [442]:
vocab_instances20_seqs_sel.seq.apply(len).unique()

array([135])

In [443]:
vocab_instances20_seqs_sel[:5]#[['vocab_pair','chr','start_pad','stop_pad']].vocab_pair.value_counts()

,vocab_pair,tot_dist,btn_dist,chr,start,stop,start_pad,stop_pad,vocab1,vocab1_start,...,vocab2_stop,tissue,loop_type,vocab_tissue_count,name,check_rs,scr_pass_rs,seq_config,seq,name_config
0,BARX2::FOXF2,28,14,chr1,206848337,206848365,206848284,206848418,BARX2,206848337,...,206848365,Colon,pro_pro,1,BARX2::FOXF2+Colon+1,0,True,A_B_seq,CTCAGCTCTCCCACAAGATCTCTGGGGAAGCTGGAAGAGGGCCACT...,BARX2::FOXF2+Colon+1+A_B
1,BARX2::FOXF2,54,40,chr4,38859601,38859655,38859561,38859695,FOXF2,38859601,...,38859655,Colon,loop_loop,2,BARX2::FOXF2+Colon+2,0,True,A_B_seq,GTGGCTCTGAGGGAATTTCAAGTAACCGTGGACTAAAGTCTTAAGA...,BARX2::FOXF2+Colon+2+A_B
2,BARX2::FOXF2,46,32,chr6,74222776,74222822,74222732,74222866,FOXF2,74222776,...,74222822,Colon,loop_loop,3,BARX2::FOXF2+Colon+3,0,True,A_B_seq,GCCACTGCACTCCAGCCTGGTGACAGAGCGAGACTCTGTCTCAAGA...,BARX2::FOXF2+Colon+3+A_B
3,BARX2::FOXF2,16,2,chr3,49384674,49384690,49384615,49384749,FOXF2,49384674,...,49384690,Colon,pro_pro,4,BARX2::FOXF2+Colon+4,0,True,A_B_seq,AGTATTATGTGCTCATGGTCCCTACCCTCCAGGGTTCACAGTGTGG...,BARX2::FOXF2+Colon+4+A_B
4,BARX2::FOXF2,16,2,chr10,112247520,112247536,112247461,112247595,BARX2,112247520,...,112247536,Colon,pro_pro,5,BARX2::FOXF2+Colon+5,0,True,A_B_seq,ATGTGGCTGCGGGGCTTTAGGAGCTGACTCTTTGGTCACGGGGAGA...,BARX2::FOXF2+Colon+5+A_B


In [444]:
# motif scaning results 
glob.glob(os.path.join(input_dir, 'motif_scan2_*.csv'))


['../data/processed/fig4_modelling/vocab_sum_final/motif_scan2_off_target_counts_df_all.csv',
 '../data/processed/fig4_modelling/vocab_sum_final/motif_scan2_motif_scan_on_target_cases_df_all.csv',
 '../data/processed/fig4_modelling/vocab_sum_final/motif_scan2_motif_scan_on_target_config_cases_df.csv']

In [445]:
off_target_res = pd.read_csv(os.path.join(input_dir, 'motif_scan2_off_target_counts_df_all.csv'),index_col=0)
off_target_res_case_2A = off_target_res[off_target_res.case=='case 2A']
print(off_target_res_case_2A.shape)
off_target_res_case_2A['off_target_pos_str'] = off_target_res_case_2A.pos_close.map(int).map(str)
off_target_res_case_2A = off_target_res_case_2A.groupby(['instance','motif']).agg({'off_target_pos_str':'|'.join}).reset_index()
off_target_res_case_2A.columns = ['off_target_name','off_target_motif','off_target_pos_str']
print(off_target_res_case_2A.shape) # groupbed by instance

(1248, 5)
(1137, 3)


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [446]:
vocab_instances20_seqs_sel.name

0        BARX2::FOXF2+Colon+1
1        BARX2::FOXF2+Colon+2
2        BARX2::FOXF2+Colon+3
3        BARX2::FOXF2+Colon+4
4        BARX2::FOXF2+Colon+5
                 ...         
16969        STAT1::UBP1+MM+6
16970        STAT1::UBP1+MM+7
16971        STAT1::UBP1+MM+8
16972        STAT1::UBP1+MM+9
16973       STAT1::UBP1+MM+10
Name: name, Length: 9560, dtype: object

In [447]:
vocab_instances20_seqs_sel_offtarget = vocab_instances20_seqs_sel.merge(off_target_res_case_2A, how='left',left_on='name',right_on='off_target_name')
vocab_instances20_seqs_sel_offtarget.off_target_name.fillna('',inplace=True)
vocab_instances20_seqs_sel_offtarget.off_target_motif.fillna('',inplace=True)
vocab_instances20_seqs_sel_offtarget.off_target_pos_str.fillna('',inplace=True)
vocab_instances20_seqs_sel_offtarget = pd.concat([vocab_instances20_seqs_sel_offtarget,vocab_instances20_seqs_sel_offtarget.apply(get_vocab_loc,axis=1)],axis=1)
vocab_instances20_seqs_sel_offtarget = vocab_instances20_seqs_sel_offtarget.groupby(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
       'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
       'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type', 'name', 'check_rs', 'scr_pass_rs', 
       'vocab_tissue_count', 'seq_config', 'seq','name_config', 'off_target_name','vocab1_start_shift', 'vocab1_stop_shift',
       'vocab2_start_shift', 'vocab2_stop_shift']).agg({'off_target_motif':'::'.join,'off_target_pos_str':'::'.join}).reset_index()

vocab_instances20_seqs_sel_offtarget = vocab_instances20_seqs_sel_offtarget.groupby(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
       'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
       'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type', 'name',
       'check_rs', 'scr_pass_rs', 'vocab_tissue_count',  'vocab1_start_shift',
       'vocab1_stop_shift', 'vocab2_start_shift', 'vocab2_stop_shift','off_target_name',
       'off_target_motif', 'off_target_pos_str']).agg({'seq_config':list,'seq':list, 'name_config':list}).reset_index()
# vocab_instances20_seqs_sel_offtarget['vocab1_loc_start'] = vocab_instances20_seqs_sel_offtarget.vocab1_start - vocab_instances20_seqs_sel_offtarget.start_pad
# vocab_instances20_seqs_sel_offtarget['vocab1_loc_stop'] = vocab_instances20_seqs_sel_offtarget.vocab1_stop - vocab_instances20_seqs_sel_offtarget.start_pad
# vocab_instances20_seqs_sel_offtarget['vocab2_loc_start'] = vocab_instances20_seqs_sel_offtarget.vocab2_start - vocab_instances20_seqs_sel_offtarget.start_pad
# vocab_instances20_seqs_sel_offtarget['vocab2_loc_stop'] = vocab_instances20_seqs_sel_offtarget.vocab2_stop - vocab_instances20_seqs_sel_offtarget.start_pad
# vocab_instances20_seqs_sel_offtarget = vocab_instances20_seqs_sel_offtarget[['']]

In [448]:
vocab_instances20_seqs_sel_offtarget.columns

Index(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
       'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
       'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type', 'name',
       'check_rs', 'scr_pass_rs', 'vocab_tissue_count', 'vocab1_start_shift',
       'vocab1_stop_shift', 'vocab2_start_shift', 'vocab2_stop_shift',
       'off_target_name', 'off_target_motif', 'off_target_pos_str',
       'seq_config', 'seq', 'name_config'],
      dtype='object')

In [449]:
vocab_instances20_seqs_sel_offtarget.seq_config.apply('|'.join).unique()
# this is the order for all configurations

array(['A_B_seq|A_scrB_seq|scrA_B_seq|scrA_scrB_seq'], dtype=object)

In [450]:
vocab_instances20_seqs_sel_offtarget.seq.apply(len).unique()
### all instances have 4 configurations

array([4])

In [451]:
print(vocab_instances20_seqs_sel_offtarget.seq.apply(lambda x: len(x[0])).unique())
print(vocab_instances20_seqs_sel_offtarget.seq.apply(lambda x: len(x[1])).unique())
print(vocab_instances20_seqs_sel_offtarget.seq.apply(lambda x: len(x[2])).unique())
print(vocab_instances20_seqs_sel_offtarget.seq.apply(lambda x: len(x[3])).unique())
### all sequences are 135 bp long

[135]
[135]
[135]
[135]


In [452]:
# row

In [453]:
# row['seq']

In [454]:
# scramble_off_target(row)

In [273]:
# row['new_seqs'] = scramble_off_target(row)
# # dict(zip(row['name_config'],row['new_seqs']))

# 2. scramble homotype off-target sequences (vocab words)

test a max of `MAX_ITER` times, 

options:
- scramble only non motif areas
- scramble the 10 bp window around the motif

## 2A get the scrambles

In [457]:
MAX_ITER = 10

In [458]:
vocab_instances20_seqs_sel_offtarget[:5]

,vocab_pair,tot_dist,btn_dist,chr,start,stop,start_pad,stop_pad,vocab1,vocab1_start,...,vocab1_start_shift,vocab1_stop_shift,vocab2_start_shift,vocab2_stop_shift,off_target_name,off_target_motif,off_target_pos_str,seq_config,seq,name_config
0,ARID5B::KLF5,17,2,chr14,69253439,69253456,69253380,69253514,KLF5,69253439,...,54,71,71,85,ARID5B::KLF5+GDSD6+8,ARID5B::KLF5,48::16,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[ACTCCCAAAACTTCCTGGGTGTAAAGGAACCAAAGGCAAGTGAGC...,"[ARID5B::KLF5+GDSD6+8+A_B, ARID5B::KLF5+GDSD6+..."
1,ARID5B::KLF5,17,2,chr6,13477440,13477457,13477381,13477515,ARID5B,13477440,...,51,64,64,81,ARID5B::KLF5+GDSD6+5,ARID5B,28,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[CAGGTGGCCTTGATCCAGGCAAAATCTGTATTAGTCAGGGTTCAC...,"[ARID5B::KLF5+GDSD6+5+A_B, ARID5B::KLF5+GDSD6+..."
2,ARID5B::KLF5,17,2,chr6,13477440,13477457,13477381,13477515,ARID5B,13477440,...,51,64,64,81,ARID5B::KLF5+GDSD6+6,ARID5B,28,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[CAGGTGGCCTTGATCCAGGCAAAATCTGTATTAGTCAGGGTTCAC...,"[ARID5B::KLF5+GDSD6+6+A_B, ARID5B::KLF5+GDSD6+..."
3,ARID5B::KLF5,18,3,chr17,7145778,7145796,7145720,7145854,ARID5B,7145778,...,50,63,63,81,,,,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAAT...,"[ARID5B::KLF5+GDSD6+10+A_B, ARID5B::KLF5+GDSD6..."
4,ARID5B::KLF5,18,3,chr19,14616989,14617007,14616931,14617065,KLF5,14616989,...,53,70,70,85,ARID5B::KLF5+GDSD6+7,KLF5,26,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[TCTTCCCCTCCACGCTGCGGGCTATGGGAAGTGGCTCGGTCAGTT...,"[ARID5B::KLF5+GDSD6+7+A_B, ARID5B::KLF5+GDSD6+..."


In [459]:
vocab_instances20_seqs_sel_offtarget.columns

Index(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
       'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
       'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type', 'name',
       'check_rs', 'scr_pass_rs', 'vocab_tissue_count', 'vocab1_start_shift',
       'vocab1_stop_shift', 'vocab2_start_shift', 'vocab2_stop_shift',
       'off_target_name', 'off_target_motif', 'off_target_pos_str',
       'seq_config', 'seq', 'name_config'],
      dtype='object')

In [474]:
#testing scrambling 
row= vocab_instances20_seqs_sel_offtarget.iloc[0,:]
print(row)
##
for x,y in zip(row['seq'],scramble_off_target(row,verbose=True)):
    print('***')
    print(x)
    print(y)
    ###


vocab_pair                                                 ARID5B::KLF5
tot_dist                                                             17
btn_dist                                                              2
chr                                                               chr14
start                                                          69253439
stop                                                           69253456
start_pad                                                      69253380
stop_pad                                                       69253514
vocab1                                                             KLF5
vocab1_start                                                   69253439
vocab1_stop                                                    69253450
vocab2                                                           ARID5B
vocab2_start                                                   69253452
vocab2_stop                                                    6

In [475]:
## iterate through the rows of the rs_filt vocab dataframe vocab_instances20_alltissue_df_rs_filt
print(vocab_instances20_seqs_sel_offtarget.shape)
vocab_instances20_seqs_sel_offtarget['scr_pass_rs'] = False
vocab_instances20_seqs_sel_offtarget['A_B_seq'] = ''
vocab_instances20_seqs_sel_offtarget['A_scrB_seq'] = ''
vocab_instances20_seqs_sel_offtarget['scrA_B_seq'] = ''
vocab_instances20_seqs_sel_offtarget['scrA_scrB_seq'] = ''

for idx, row in vocab_instances20_seqs_sel_offtarget.iterrows():
    ## DEBUG
    if (idx%100)==0:
        print(idx)
    
    # set default
    scr_pass_rs = False
    num_iter = 0
    # iterate scrambles until find some that pass the check_rs filter
    while (not scr_pass_rs) ^ (num_iter > MAX_ITER):
        # array(['A_B_seq|A_scrB_seq|scrA_B_seq|scrA_scrB_seq'], dtype=object), is order of results
        A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq = scramble_off_target(row)



        # check_rs
        A_B_seq_rs = check_rs(A_B_seq)
        A_scrB_seq_rs = check_rs(A_scrB_seq)
        scrA_B_seq_rs = check_rs(scrA_B_seq)
        scrA_scrB_seq_rs = check_rs(scrA_scrB_seq)

        if (A_B_seq_rs+A_scrB_seq_rs+scrA_B_seq_rs+scrA_scrB_seq_rs==0):
            scr_pass_rs=True
        num_iter+=1
    
    # add information to df
    vocab_instances20_seqs_sel_offtarget.at[idx, 'scr_pass_rs'] = scr_pass_rs
    vocab_instances20_seqs_sel_offtarget.at[idx, 'A_B_seq'] = A_B_seq
    vocab_instances20_seqs_sel_offtarget.at[idx, 'A_scrB_seq'] = A_scrB_seq
    vocab_instances20_seqs_sel_offtarget.at[idx, 'scrA_B_seq'] = scrA_B_seq
    vocab_instances20_seqs_sel_offtarget.at[idx, 'scrA_scrB_seq'] = scrA_scrB_seq
    



(2390, 30)
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


## 2B some gut checks and saving

In [476]:
# manual step
vocab_instances20_seqs_sel_offtarget.scr_pass_rs.value_counts()

True    2390
Name: scr_pass_rs, dtype: int64

all sequences and scr configurations pass the rs filter

if they don't keep running the scrambling until they all pass

In [477]:
vocab_instances20_seqs_sel_offtarget.to_csv(os.path.join(save_dir, 'vocab_instances20_seqs_sel_offtarget_all_col.csv'))

In [478]:
print(vocab_instances20_seqs_sel_offtarget.columns)
vocab_instances20_seqs_sel_offtarget[:5]

Index(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
       'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
       'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type', 'name',
       'check_rs', 'scr_pass_rs', 'vocab_tissue_count', 'vocab1_start_shift',
       'vocab1_stop_shift', 'vocab2_start_shift', 'vocab2_stop_shift',
       'off_target_name', 'off_target_motif', 'off_target_pos_str',
       'seq_config', 'seq', 'name_config', 'A_B_seq', 'A_scrB_seq',
       'scrA_B_seq', 'scrA_scrB_seq'],
      dtype='object')


,vocab_pair,tot_dist,btn_dist,chr,start,stop,start_pad,stop_pad,vocab1,vocab1_start,...,off_target_name,off_target_motif,off_target_pos_str,seq_config,seq,name_config,A_B_seq,A_scrB_seq,scrA_B_seq,scrA_scrB_seq
0,ARID5B::KLF5,17,2,chr14,69253439,69253456,69253380,69253514,KLF5,69253439,...,ARID5B::KLF5+GDSD6+8,ARID5B::KLF5,48::16,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[ACTCCCAAAACTTCCTGGGTGTAAAGGAACCAAAGGCAAGTGAGC...,"[ARID5B::KLF5+GDSD6+8+A_B, ARID5B::KLF5+GDSD6+...",ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...
1,ARID5B::KLF5,17,2,chr6,13477440,13477457,13477381,13477515,ARID5B,13477440,...,ARID5B::KLF5+GDSD6+5,ARID5B,28,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[CAGGTGGCCTTGATCCAGGCAAAATCTGTATTAGTCAGGGTTCAC...,"[ARID5B::KLF5+GDSD6+5+A_B, ARID5B::KLF5+GDSD6+...",CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...
2,ARID5B::KLF5,17,2,chr6,13477440,13477457,13477381,13477515,ARID5B,13477440,...,ARID5B::KLF5+GDSD6+6,ARID5B,28,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[CAGGTGGCCTTGATCCAGGCAAAATCTGTATTAGTCAGGGTTCAC...,"[ARID5B::KLF5+GDSD6+6+A_B, ARID5B::KLF5+GDSD6+...",CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...
3,ARID5B::KLF5,18,3,chr17,7145778,7145796,7145720,7145854,ARID5B,7145778,...,,,,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAAT...,"[ARID5B::KLF5+GDSD6+10+A_B, ARID5B::KLF5+GDSD6...",GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...
4,ARID5B::KLF5,18,3,chr19,14616989,14617007,14616931,14617065,KLF5,14616989,...,ARID5B::KLF5+GDSD6+7,KLF5,26,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[TCTTCCCCTCCACGCTGCGGGCTATGGGAAGTGGCTCGGTCAGTT...,"[ARID5B::KLF5+GDSD6+7+A_B, ARID5B::KLF5+GDSD6+...",TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...


In [479]:
print(vocab_instances20_seqs_sel_offtarget.A_B_seq.apply(len).unique())
print(vocab_instances20_seqs_sel_offtarget.A_scrB_seq.apply(len).unique())
print(vocab_instances20_seqs_sel_offtarget.scrA_B_seq.apply(len).unique())
print(vocab_instances20_seqs_sel_offtarget.scrA_scrB_seq.apply(len).unique())

# all sequences are 135 bp in length

[135]
[135]
[135]
[135]


In [480]:
row = vocab_instances20_seqs_sel_offtarget.iloc[0,:]
print(row)
print(row['seq'][0])
print(row['A_B_seq'])
print(row['A_scrB_seq'])
print(row['scrA_B_seq'])
print(row['scrA_scrB_seq'])


vocab_pair                                                 ARID5B::KLF5
tot_dist                                                             17
btn_dist                                                              2
chr                                                               chr14
start                                                          69253439
stop                                                           69253456
start_pad                                                      69253380
stop_pad                                                       69253514
vocab1                                                             KLF5
vocab1_start                                                   69253439
vocab1_stop                                                    69253450
vocab2                                                           ARID5B
vocab2_start                                                   69253452
vocab2_stop                                                    6

In [532]:
vocab_instances20_seqs_sel_offtarget

,vocab_pair,tot_dist,btn_dist,chr,start,stop,start_pad,stop_pad,vocab1,vocab1_start,...,off_target_name,off_target_motif,off_target_pos_str,seq_config,seq,name_config,A_B_seq,A_scrB_seq,scrA_B_seq,scrA_scrB_seq
0,ARID5B::KLF5,17,2,chr14,69253439,69253456,69253380,69253514,KLF5,69253439,...,ARID5B::KLF5+GDSD6+8,ARID5B::KLF5,48::16,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[ACTCCCAAAACTTCCTGGGTGTAAAGGAACCAAAGGCAAGTGAGC...,"[ARID5B::KLF5+GDSD6+8+A_B, ARID5B::KLF5+GDSD6+...",ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...
1,ARID5B::KLF5,17,2,chr6,13477440,13477457,13477381,13477515,ARID5B,13477440,...,ARID5B::KLF5+GDSD6+5,ARID5B,28,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[CAGGTGGCCTTGATCCAGGCAAAATCTGTATTAGTCAGGGTTCAC...,"[ARID5B::KLF5+GDSD6+5+A_B, ARID5B::KLF5+GDSD6+...",CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...
2,ARID5B::KLF5,17,2,chr6,13477440,13477457,13477381,13477515,ARID5B,13477440,...,ARID5B::KLF5+GDSD6+6,ARID5B,28,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[CAGGTGGCCTTGATCCAGGCAAAATCTGTATTAGTCAGGGTTCAC...,"[ARID5B::KLF5+GDSD6+6+A_B, ARID5B::KLF5+GDSD6+...",CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...
3,ARID5B::KLF5,18,3,chr17,7145778,7145796,7145720,7145854,ARID5B,7145778,...,,,,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAAT...,"[ARID5B::KLF5+GDSD6+10+A_B, ARID5B::KLF5+GDSD6...",GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...
4,ARID5B::KLF5,18,3,chr19,14616989,14617007,14616931,14617065,KLF5,14616989,...,ARID5B::KLF5+GDSD6+7,KLF5,26,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[TCTTCCCCTCCACGCTGCGGGCTATGGGAAGTGGCTCGGTCAGTT...,"[ARID5B::KLF5+GDSD6+7+A_B, ARID5B::KLF5+GDSD6+...",TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,ZFX::E2F6,26,4,chr20,2632854,2632880,2632800,2632934,ZFX,2632854,...,,,,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[CCGAGTAGGGGACGGGAACGCCTGGGTCCCGGGGACGTGGGGCGC...,"[ZFX::E2F6+SCC+8+A_B, ZFX::E2F6+SCC+8+A_scrB, ...",CCGAGTAGGGGACGGGAACGCCTGGGTCCCGGGGACGTGGGGCGCA...,CCGAGTAGGGGACGGGAACGCCTGGGTCCCGGGGACGTGGGGCGCA...,CCGAGTAGGGGACGGGAACGCCTGGGTCCCGGGGACGTGGGGCGCA...,CCGAGTAGGGGACGGGAACGCCTGGGTCCCGGGGACGTGGGGCGCA...
2386,ZFX::E2F6,28,6,chr2,85811652,85811680,85811599,85811733,ZFX,85811652,...,ZFX::E2F6+SCC+1,ZFX,104|104,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[GCGGCGGCAGCAGCGATGGTGAGGGCCCAGGCGGGGCCGGCCAGC...,"[ZFX::E2F6+SCC+1+A_B, ZFX::E2F6+SCC+1+A_scrB, ...",GCGGCGGCAGCAGCGATGGTGAGGGCCCAGGCGGGGCCGGCCAGCC...,GCGGCGGCAGCAGCGATGGTGAGGGCCCAGGCGGGGCCGGCCAGCC...,GCGGCGGCAGCAGCGATGGTGAGGGCCCAGGCGGGGCCGGCCAGCC...,GCGGCGGCAGCAGCGATGGTGAGGGCCCAGGCGGGGCCGGCCAGCC...
2387,ZFX::E2F6,28,6,chr2,220083840,220083868,220083787,220083921,E2F6,220083840,...,ZFX::E2F6+SCC+2,E2F6,110,"[A_B_seq, A_scrB_seq, scrA_B_seq, scrA_scrB_seq]",[GTGCAGGCAACGAATGCAAATTAGAGAAGCAGAGCCGGCTTGGTC...,"[ZFX::E2F6+SCC+2+A_B, ZFX::E2F6+SCC+2+A_scrB, ...",GTGCAGGCAACGAATGCAAATTAGAGAAGCAGAGCCGGCTTGGTCA...,GTGCAGGCAACGAATGCAAATTAGAGAAGCAGAGCCGGCTTGGTCA...,GTGCAGGCAACGAATGCAAATTAGAGAAGCAGAGCCGGCTTGGTCA...,GTGCAGGCAACGAATGCAAATTAGAGAAGCAGAGCCGGCTTGGTCA...
2388,ZFX::E2F6,28,6,chr7,44153086,44153114,44153033

In [549]:
off_target_num_df = pd.DataFrame(vocab_instances20_seqs_sel_offtarget.set_index('name').off_target_pos_str.apply(off_target_pos_str_find_num)).reset_index()
off_target_num_df['instance'] = off_target_num_df.name.apply(lambda x: '+'.join(x.split('+')[:-1]))
off_target_num_df_counts = off_target_num_df[off_target_num_df.off_target_pos_str==0].instance.value_counts().value_counts().sort_index()
print(off_target_num_df_counts/off_target_num_df_counts.sum())
off_target_num_df[off_target_num_df.off_target_pos_str==0].instance.value_counts().describe()

1     0.004184
2     0.012552
3     0.033473
4     0.100418
5     0.221757
6     0.188285
7     0.205021
8     0.125523
9     0.066946
10    0.041841
Name: instance, dtype: float64


count    239.000000
mean       6.230126
std        1.780311
min        1.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       10.000000
Name: instance, dtype: float64

In [288]:
# vocab_instances20_seqs_sel_offtarget[vocab_instances20_seqs_sel_offtarget.scrA_B_seq.apply(len)==249]

In [483]:
vocab_instances20_seqs_sel_offtarget.reset_index(drop=True, inplace=True)
# vocab_instances20_seqs_sel_offtarget[ 'vocab_tissue_count'] = vocab_instances20_seqs_sel_offtarget.groupby(['vocab_pair','tissue']).cumcount()+1
# vocab_instances20_seqs_sel_offtarget['name'] = vocab_instances20_seqs_sel_offtarget.vocab_pair + '+'+ vocab_instances20_seqs_sel_offtarget.tissue + '+'+ vocab_instances20_seqs_sel_offtarget.vocab_tissue_count.map(str)
vocab_instances20_seqs = pd.melt(vocab_instances20_seqs_sel_offtarget.drop(['start','stop', 'vocab1_start', 'vocab1_stop',
                                                                            'vocab2_start', 'vocab2_stop','off_target_name', 'off_target_motif', 'off_target_pos_str',
                                                                            'seq_config','seq','name_config'],axis=1),
         id_vars=['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start_pad', 'stop_pad',
                  'vocab1','vocab1_start_shift','vocab1_stop_shift','vocab2', 'vocab2_start_shift', 'vocab2_stop_shift',
                  'tissue', 'loop_type',  'vocab_tissue_count','name','check_rs','scr_pass_rs'], 
            var_name='seq_config',
             value_name="seq")
vocab_instances20_seqs['name_config'] = vocab_instances20_seqs.name + '+' + vocab_instances20_seqs.seq_config.str.split('_seq',expand=True).iloc[:,0]

vocab_instances20_seqs.sort_values(['name_config'])

,vocab_pair,tot_dist,btn_dist,chr,start_pad,stop_pad,vocab1,vocab1_start_shift,vocab1_stop_shift,vocab2,...,vocab2_stop_shift,tissue,loop_type,vocab_tissue_count,name,check_rs,scr_pass_rs,seq_config,seq,name_config
7,ARID5B::KLF5,21,6,chr18,61243074,61243208,ARID5B,49,62,KLF5,...,83,GDSD6,loop_loop,1,ARID5B::KLF5+GDSD6+1,0,True,A_B_seq,AATCCAGGTTTTAAGGATTACACCATAGCACTCCTAAAGATACATT...,ARID5B::KLF5+GDSD6+1+A_B
2397,ARID5B::KLF5,21,6,chr18,61243074,61243208,ARID5B,49,62,KLF5,...,83,GDSD6,loop_loop,1,ARID5B::KLF5+GDSD6+1,0,True,A_scrB_seq,AATCCAGGTTTTAAGGATTACACCATAGCACTCCTAAAGATACATT...,ARID5B::KLF5+GDSD6+1+A_scrB
4787,ARID5B::KLF5,21,6,chr18,61243074,61243208,ARID5B,49,62,KLF5,...,83,GDSD6,loop_loop,1,ARID5B::KLF5+GDSD6+1,0,True,scrA_B_seq,AATCCAGGTTTTAAGGATTACACCATAGCACTCCTAAAGATACATT...,ARID5B::KLF5+GDSD6+1+scrA_B
7177,ARID5B::KLF5,21,6,chr18,61243074,61243208,ARID5B,49,62,KLF5,...,83,GDSD6,loop_loop,1,ARID5B::KLF5+GDSD6+1,0,True,scrA_scrB_seq,AATCCAGGTTTTAAGGATTACACCATAGCACTCCTAAAGATACATT...,ARID5B::KLF5+GDSD6+1+scrA_scrB
3,ARID5B::KLF5,18,3,chr17,7145720,7145854,ARID5B,50,63,KLF5,...,81,GDSD6,loop_loop,10,ARID5B::KLF5+GDSD6+10,0,True,A_B_seq,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,ARID5B::KLF5+GDSD6+10+A_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9555,ZFX::E2F6,26,4,chr20,2632800,2632934,ZFX,50,68,E2F6,...,86,SCC,loop_loop,8,ZFX::E2F6+SCC+8,0,True,scrA_scrB_seq,CCGAGTAGGGGACGGGAACGCCTGGGTCCCGGGGACGTGGGGCGCA...,ZFX::E2F6+SCC+8+scrA_scrB
2388,ZFX::E2F6,28,6,chr7,44153033,44153167,E2F6,47,68,ZFX,...,85,SCC,pro_pro,9,ZFX::E2F6+SCC+9,0,True,A_B_seq,AGGTAGAGTCTGGGGAGCCTGGGGGCGAAATTCAGAGAGGGAGGGC...,ZFX::E2F6+SCC+9+A_B
4778,ZFX::E2F6,28,6,chr7,44153033,44153167,E2F6,47,68,ZFX,...,85,SCC,pro_pro,9,ZFX::E2F6+SCC+9,0,True,A_scrB_seq,AGGTAGAGTCTGGGGAGCCTGGGGGCGAAATTCAGAGAGGGAGGGC...,ZFX::E2F6+SCC+9+A_scrB
7168,ZFX::E2F6,28,6,chr7,44153033,44153167,E2F6,47,68,ZFX,...,85,SCC,pro_pro,9,ZFX::E2F6+SCC+9,0,True,scrA_B_seq,AGGTAGAGTCTGGGGAGCCTGGGGGCGAAATTCAGAGAGGGAGGGC...,ZFX::E2F6+SCC+9+scrA_B


In [499]:
vocab_instances20_seqs.seq.str.len().value_counts()

135    9560
Name: seq, dtype: int64

In [484]:
vocab_instances20_seqs.name_config.unique().shape

(9560,)

In [485]:
vocab_instances20_seqs[['name','seq_config']].drop_duplicates().shape

(9560, 2)

In [486]:
vocab_instances20_seqs_sel.seq_config.unique()

array(['A_B_seq', 'scrA_B_seq', 'A_scrB_seq', 'scrA_scrB_seq'],
      dtype=object)

In [487]:
vocab_instances20_seqs.seq.apply(len).unique()

array([135])

In [488]:
# save the vocab info
print(vocab_instances20_seqs.shape, vocab_instances20_alltissue_df_rs_filt.shape)
vocab_instances20_seqs.to_csv(os.path.join(save_dir, 'vocab_instances20_seq_wscr.csv'))

(9560, 21) (4246, 20)


135    9560
Name: seq, dtype: int64

In [489]:
vocab_instances20_seqs.columns


Index(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start_pad', 'stop_pad',
       'vocab1', 'vocab1_start_shift', 'vocab1_stop_shift', 'vocab2',
       'vocab2_start_shift', 'vocab2_stop_shift', 'tissue', 'loop_type',
       'vocab_tissue_count', 'name', 'check_rs', 'scr_pass_rs', 'seq_config',
       'seq', 'name_config'],
      dtype='object')

# 3. make post scan mpra library

Input DFs:
- `vocab_instances20_seqs`: `vocab_instances20_seq_wscr.csv`
- `pos_genes_df_rs_filt`: `pos_genes_position_wseq_rsfilt.csv`
- `neg_genes_df_rs_filt`: `neg_genes_position_wseq_rsfilt.csv`

Output DF: 
- `mpra_lib_seq`: 'mpra_lib_seq.csv`

In [490]:
vocab_instances20_seqs[['vocab_pair','tissue']].drop_duplicates().shape

(239, 2)

still have 239 vocab words

In [491]:
vocab_instances20_seqs[vocab_instances20_seqs.vocab_tissue_count==10].vocab_pair.unique().shape

(239,)

all 239 vocab words have at least 10 genomic instances

In [493]:
vocab_instances20_seqs_sel[vocab_instances20_seqs_sel.seq.apply(len)!=135]

,vocab_pair,tot_dist,btn_dist,chr,start,stop,start_pad,stop_pad,vocab1,vocab1_start,...,vocab2_stop,tissue,loop_type,vocab_tissue_count,name,check_rs,scr_pass_rs,seq_config,seq,name_config


all vocab genomic i

### saving selection of vocab words
how `name_config` works:

`df['vocab_pair','tissue','vocab_tissue_count','seq_config'] = df.name_config.str.split('+',expand=True)`

In [596]:
vocab_instances20_seqs[['name','seq_config']].drop_duplicates().shape

(9560, 2)

In [597]:
vocab_instances20_seqs.name.value_counts()

SNAI2::WT1+Colon+7             4
SP4::THAP1+Colon+8             4
NFE2L1::TBX15+MM+6             4
NFE2L1::NR3C1+MM+8             4
ARID5B::STAT1+MM+2             4
                              ..
KLF15::ZNF219+Melanocytes+2    4
RORA::TP63+GDSD6+2             4
EGR2::TP63+GDSD6+4             4
EN2::PRRX1+Melanocytes+10      4
KLF15::KLF5+Colon+9            4
Name: name, Length: 2390, dtype: int64

In [598]:
vocab_instances20_seqs.columns

Index(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start_pad', 'stop_pad',
       'vocab1', 'vocab1_start_shift', 'vocab1_stop_shift', 'vocab2',
       'vocab2_start_shift', 'vocab2_stop_shift', 'tissue', 'loop_type',
       'vocab_tissue_count', 'name', 'check_rs', 'scr_pass_rs', 'seq_config',
       'seq', 'name_config'],
      dtype='object')

In [601]:
vocab_instances20_seqs[:5]

,vocab_pair,tot_dist,btn_dist,chr,start_pad,stop_pad,vocab1,vocab1_start_shift,vocab1_stop_shift,vocab2,...,vocab2_stop_shift,tissue,loop_type,vocab_tissue_count,name,check_rs,scr_pass_rs,seq_config,seq,name_config
0,ARID5B::KLF5,17,2,chr14,69253380,69253514,KLF5,54,71,ARID5B,...,85,GDSD6,loop_loop,8,ARID5B::KLF5+GDSD6+8,0,True,A_B_seq,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,ARID5B::KLF5+GDSD6+8+A_B
1,ARID5B::KLF5,17,2,chr6,13477381,13477515,ARID5B,51,64,KLF5,...,81,GDSD6,loop_loop,5,ARID5B::KLF5+GDSD6+5,0,True,A_B_seq,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,ARID5B::KLF5+GDSD6+5+A_B
2,ARID5B::KLF5,17,2,chr6,13477381,13477515,ARID5B,51,64,KLF5,...,81,GDSD6,pro_pro,6,ARID5B::KLF5+GDSD6+6,0,True,A_B_seq,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,ARID5B::KLF5+GDSD6+6+A_B
3,ARID5B::KLF5,18,3,chr17,7145720,7145854,ARID5B,50,63,KLF5,...,81,GDSD6,loop_loop,10,ARID5B::KLF5+GDSD6+10,0,True,A_B_seq,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,ARID5B::KLF5+GDSD6+10+A_B
4,ARID5B::KLF5,18,3,chr19,14616931,14617065,KLF5,53,70,ARID5B,...,85,GDSD6,loop_loop,7,ARID5B::KLF5+GDSD6+7,0,True,A_B_seq,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,ARID5B::KLF5+GDSD6+7+A_B


In [602]:
vocab_instances20_seqs.name_config.unique().shape

(9560,)

In [603]:
vocab_instances20_seqs_selcols = vocab_instances20_seqs[['chr','start_pad','stop_pad','name_config','seq']]
vocab_instances20_seqs_selcols.columns = ['chr','start','stop','name','seq']
vocab_instances20_seqs_selcols['seq_type'] = 'vocab'

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [604]:
pos_genes_df_seqs_sel_cols = pos_genes_df_rs_filt[['chr','start','end','gene','sequence']]
pos_genes_df_seqs_sel_cols.columns = ['chr','start','stop','name','seq']
pos_genes_df_seqs_sel_cols['seq_type'] = 'pos'
neg_genes_df_seqs_sel_cols = neg_genes_df_rs_filt[['chr','start','end','gene','sequence']]
neg_genes_df_seqs_sel_cols.columns = ['chr','start','stop','name','seq']
neg_genes_df_seqs_sel_cols['seq_type'] = 'neg'


In [605]:
mpra_lib_seq = pd.concat([vocab_instances20_seqs_selcols,pos_genes_df_seqs_sel_cols, neg_genes_df_seqs_sel_cols ],axis=0,sort=True)
mpra_lib_seq

,chr,name,seq,seq_type,start,stop
0,chr14,ARID5B::KLF5+GDSD6+8+A_B,ACTCCCAAAACTTCCTAACAGAAGAGCGGTTGAAGGCAAGTGAGCT...,vocab,69253380,69253514
1,chr6,ARID5B::KLF5+GDSD6+5+A_B,CAGGTGGCCTTGATCCAGGCAAAATCTGGTTAAGATTCGGTTCACC...,vocab,13477381,13477515
2,chr6,ARID5B::KLF5+GDSD6+6+A_B,CAGGTGGCCTTGATCCAGGCAAAATCTGAGTGGTAGTTCATTCACC...,vocab,13477381,13477515
3,chr17,ARID5B::KLF5+GDSD6+10+A_B,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,vocab,7145720,7145854
4,chr19,ARID5B::KLF5+GDSD6+7+A_B,TCTTCCCCTCCACGCTGCGGGCTATGGGTACGCAGGGTATGCGTTT...,vocab,14616931,14617065
...,...,...,...,...,...,...
89,chrX,RHOXF2B,ACCACGCTCTCCTAGGCAGTGCTAGAACTGTGAGTCGCCGCCTGCA...,neg,119211572,119211707
90,chrX,CT47A12,AGTCTCTCTCTTTATTGAGGGAATAAGAGTCTGTCTCAGACGACAC...,neg,120066016,120066151
91,chrX,CT47A6,AGTCTCTCTCTTTATTGAGGGAATAAGAGTCTGTCTCAGACGACAC...,neg,120095202,120095337
92,chrX,SPANXN4,TGTCAGAGGTGGTTCTTTGTGATGCCAAAGCCACCCTAGGACTACC...,neg,142113569,142113704


#### some stats then saving

In [606]:
mpra_lib_seq.seq_type.value_counts()

vocab    9560
neg        89
pos        72
Name: seq_type, dtype: int64

In [607]:
vocab_instances20_seqs_sel.tissue.value_counts()/40

GDSD6          49.0
SCC            43.0
Colon          42.0
MM             40.0
GM12878        39.0
Melanocytes    26.0
Name: tissue, dtype: float64

In [608]:
# vocab_instances20_seqs_sel.vocab_pair[vocab_instances20_seqs_sel.vocab_pair.isin(irf_vocab)].unique()

In [609]:
# # number of vocab words that have irf vocab setup
# vocab_instances20_seqs_sel.vocab_pair[vocab_instances20_seqs_sel.vocab_pair.isin(irf_vocab)].unique().shape


In [610]:
mpra_lib_seq.to_csv(os.path.join(save_dir, 'mpra_lib_seq.csv'))

# 4. add MPRA adaptors and barcodes

done in 8E1R (based on `make_library_psych-postcadd-01119.ipynb` from the mpra library

note that the rs sites chosen from section 4B are: `[RS_ECORI ,RS_BAMHI , RS_XHOI , RS_NHEI]`

input is `../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_lib_seq.csv`

output is `../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_oligo_df_filt.csv`

In [611]:

# define constants
LEN_FILLER = 20
NUM_OLIGO_PER_MOTIF = 10
DESIRED_OLIGO_LENGTH = 135 #(changed from 150 )
MAX_OLIGO_LENGTH = 230
MAX_MOTIF_LENGTH = 63
FORWARD_PCR_PRIMER = 'ACTGGCCGCTTCACTG'
REVERSE_PCR_PRIMER = 'AGATCGGAAGAGCGTCG'
# LINKER_SPACER_v10 = 'CAGTATGCGATGCTCATGATTGTGACTGAAAGTCTATGAGTCCATTGA'
RS_ECORI = 'GAATTC' # 5'-3'
RS_BAMHI = 'GGATCC'
RS_XHOI = 'CTCGAG'
RS_NHEI = 'GCTAGC'


RS_LIST = [RS_ECORI ,RS_BAMHI , RS_XHOI , RS_NHEI]

LETTERS=['A', 'C', 'G', 'T']

# 7. MPRA final check 

- checks to make sure all sequences have 10 barcodes
- all rs sites are passed (done in R notebook)
- that there are still 239 vocab words with 40 instance (4 config of 10 each)

- `../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_lib_seq.csv` - includes information on all the sequences used
- `../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_oligo_df_filt.csv` - oligo information with a lot of columns (RS site check results etc.(
- `../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_oligo_df_final.txt` - with columns: name, seq_type, id, oligo, seq, filler, barcode
- `../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_oligo_df_final_SUBMIT.txt` - with columns: id, oligo

In [612]:
glob.glob(os.path.join(save_dir))

['../data/processed/fig4_modelling/vocab_sum_final_post_scan/']

In [613]:
oligo_df_filt = pd.read_csv(os.path.join(save_dir,'mpra_oligo_df_filt.csv'),index_col=0)
print(oligo_df_filt.shape)
oligo_df_filt[:5]

(97210, 21)


,name,seq,seq_type,count,id,pass_restrict_seq,pass_restrict_fwd_seq,pass_restrict_seq_xho,barcode,len_seq,...,1:n(),filler,filler_check,oligo,len_oligo,pass_restrict_ECORI,pass_restrict_BAMHI,pass_restrict_XHOI,pass_restrict_NHEI,pass_all
1,ABCF3,ACCTTGAGCTCCCAGCCTTTGCCCCACCCCTCGCTCCCGGAACTCC...,pos,1,ABCF3.1,True,True,True,AGTTGGAATGAAGGCCTCGA,136,...,4,TCCTTACAAAAGAACGGCCG,True,ACTGGCCGCTTCACTGACCTTGAGCTCCCAGCCTTTGCCCCACCCC...,221,True,True,True,True,True
2,ABCF3,ACCTTGAGCTCCCAGCCTTTGCCCCACCCCTCGCTCCCGGAACTCC...,pos,2,ABCF3.2,True,True,True,GTCATAGTCAGAGAAGCTCT,136,...,3,TCTCTTGACGCGATGCGAGC,True,ACTGGCCGCTTCACTGACCTTGAGCTCCCAGCCTTTGCCCCACCCC...,221,True,True,True,True,True
3,ABCF3,ACCTTGAGCTCCCAGCCTTTGCCCCACCCCTCGCTCCCGGAACTCC...,pos,3,ABCF3.3,True,True,True,GATACTTCATGCAATATGCT,136,...,19,TGCTATGGGGGATAGATCTT,True,ACTGGCCGCTTCACTGACCTTGAGCTCCCAGCCTTTGCCCCACCCC...,221,True,True,True,True,True
4,ABCF3,ACCTTGAGCTCCCAGCCTTTGCCCCACCCCTCGCTCCCGGAACTCC...,pos,4,ABCF3.4,True,True,True,CGATTGCAGGAGAATAGTCC,136,...,2,ACCGTGCCAGGGCTCACCGT,True,ACTGGCCGCTTCACTGACCTTGAGCTCCCAGCCTTTGCCCCACCCC...,221,True,True,True,True,True
5,ABCF3,ACCTTGAGCTCCCAGCCTTTGCCCCACCCCTCGCTCCCGGAACTCC...,pos,5,ABCF3.5,True,True,True,TGAGAACCAGTTCCAACGCA,136,...,11,AGACTGGTAGTGAACCTGAT,True,ACTGGCCGCTTCACTGACCTTGAGCTCCCAGCCTTTGCCCCACCCC...,221,True,True,True,True,True


In [614]:
seq_counts = oligo_df_filt.name.value_counts()
seq_counts[seq_counts!=10]

Series([], Name: name, dtype: int64)

 all sequences have 10 barcodes


rs sites checks (done in R notebook)
- no EcoRI 
- no BamHI sites
- exactly 1 XhoI site
- exactly 1 NheI site


In [615]:
vocab_oligo_df =  oligo_df_filt[oligo_df_filt.seq_type=='vocab']
    
vocab_oligo_df[['vocab_pair','tissue','vocab_tissue_count','seq_config']]= vocab_oligo_df.name.str.split('+',expand=True)

In [616]:
vocab_oligo_counts = vocab_oligo_df.vocab_pair.value_counts()
vocab_oligo_counts[vocab_oligo_counts!=400]

Series([], Name: vocab_pair, dtype: int64)

 all vocab types have 4 configs x 10 genomic instances x 10 barcodes = 400 oligos


In [617]:
vocab_oligo_counts.shape

(239,)

there are 239 vocab words

## 7B motif scan

### 1. make fasta 


In [619]:
fasta_file = os.path.join(save_dir, 'mpra_vocab_seq5.fasta')
print(vocab_oligo_df[['name','seq']].drop_duplicates().shape)
with open(fasta_file, 'w') as f:
    for idx, row in vocab_oligo_df[['name','seq']].drop_duplicates().iterrows():
        f.write('>'+row['name']+'\n')
        f.write(row['seq']+'\n')

(9560, 2)


In [ ]:
fasta_control_file = os.path.join(save_dir, 'mpra_control_seq.fasta')
control_oligo_df = oligo_df_filt[oligo_df_filt.seq_type.isin(['pos','neg'])]
print(control_oligo_df[['name','seq']].drop_duplicates().shape)
with open(fasta_control_file, 'w') as f:
    for idx, row in control_oligo_df[['name','seq']].drop_duplicates().iterrows():
        f.write('>'+row['name']+'\n')
        f.write(row['seq']+'\n')

### scanning try 3 MOODS package
https://github.com/jhkorhonen/MOODS/wiki/Getting-started
Matrices. The matrices are expected in the JASPAR .pfm format or the .adm format for first-order models. A JASPAR .pfm file consist of four rows, specifying the counts or frequencies for nucleotides A, C, G and T, respectively:

#### DESCRIPTION

MOODS 1.9.1 introduced a Python command-line script moods-dna.py for standalone PWM analysis using the MOODS algorithms. This section explains the basics of using `moods-dna.py`

#### Input matrics (motifs)
pfms are in `/Users/mguo123/Google Drive/1_khavari/omics_project-LD/pan_omics/data/external/hocomoco_human_trim`
Note: the extensions are pwm but are pfms

#### Input sequences
Sequences. The sequences can be given either as plain text files, or as fasta files possibly containing multiple sequences. Newlines and leading and trailing whitespace will be ignored, but other characters not encoding nucleotides will be treated as non-matchable positions (i.e., they should not appear outside fasta headers). IUPAC nucleotide ambiguity codes (WSMKRYBDHV) will be treated as coding SNPs, and MOODS will match the matrices versus all possible combinations.

#### Output (csv)
The comma-separated fields are:

- Sequence name (either file name or fasta header).
- Matrix file name.
- Hit position (first position of sequence is position 0).
- Indicates whether the match is versus the input strand (+) or the reverse complement (-).
- Match score.
- Hit site in the original sequence (note that if the match is -, then the match is versus the reverse complement of this).
- Hit sequence with SNPs applied, i.e. shows how the ambiguity codes are resolved to get this score (there can be multiple hits at the same position with different "real" hit sequences).


#### Installation 

follow: https://pypi.org/project/MOODS-python/

this doesn't work with subprocess
```
pip install --user MOODS-python
PATH="$PATH:`python -m site --user-base`/bin"
```

#### Basic usage
The basic usage pattern of `moods-dna.py` is the following:

```
moods-dna.py --help
usage: moods-dna.py [-h] [-v] [-m M [M ...]] [-S M [M ...]] [-s S [S ...]]
                    [-p p] [-t T] [-B n] [-o outfile] [--sep S] [-R]
                    [--no-snps] [--batch] [--bg pA pC pG pT] [--ps p]
                    [--log-base x] [--lo-bg pA pC pG pT]
                    [--threshold-precision x]

matches position weight matrices against DNA sequences

optional arguments:
  -h, --help            show this help message and exit
  -v, --verbosity       verbose (-vv, -vvv for more)

input files (at least one matrix and sequence file required):
  -m M [M ...], --matrices M [M ...]
                        matrix files (count/frequency, will be converted to
                        PWM before matching)
  -S M [M ...], --score-matrices M [M ...]
                        matrix files (PWM/other score matrix, will be matched
                        directly)
  -s S [S ...], --sequences S [S ...]
                        sequence files

threshold selection (exactly one required):
  -p p, --p-value p     compute threshold from p-value
  -t T, --threshold T   use specified absolute threshold
  -B n, --best-hits n   return at least the specified amount of best matches

output (optional):
  -o outfile, --outfile outfile
                        output to file instead of standard output
  --sep S               set field separator in out (default ",")

search and model behaviour (optional):
  -R, --no-rc           disable matching versus reverse complement strand
  --no-snps             ignore IUPAC symbols coding multiple nucleotides
  --batch               do not recompute thresholds from p-values for each
                        sequence separately (recommended when dealing with
                        lots of short sequences)
  --bg pA pC pG pT      background distribution for computing thresholds from
                        p-value with --batch (default is 0.25 for all alleles)
  --ps p                total pseudocount added to each matrix column in log-
                        odds conversion (default = 0.01)
  --log-base x          logarithm base for log-odds conversion (default
                        natural logarithm)
  --lo-bg pA pC pG pT   background distribution for log-odds conversion
                        (default is 0.25 for all alleles)
  --threshold-precision x
                        specify the precision used for computing the
                        thresholds from p-values (default = 2000.0)
                        
                        
moods-dna.py -m example-data/matrices/*.{pfm,adm} -s example-data/seq/chr1-5k-55k.fa -p 0.0001
moods-dna.py -m ../data/external/hocomoco_human_trim/*.pwm -s ../data/processed/fig4_modelling/vocab_sum_final/mpra_vocab_seq.fasta -p 0.0001

moods-dna.py -m ../data/external/hocomoco_human_trim/FOXM1_HUMAN.H11MO.0.A.pwm.trim.pwm -s ../data/processed/fig4_modelling/vocab_sum_final/mpra_vocab_seq.fasta -p 0.0001


```

In [510]:
# # !pip install --user MOODS-python
# !PATH="$PATH:`python -m site --user-base`/bin"
# glob.glob(os.path.join(save_dir,"*fasta"))
import subprocess

In [511]:
# num actual tfs used 
list_tfs = sorted(set(list(vocab_instances20_seqs.vocab1.unique()) + list(vocab_instances20_seqs.vocab2.unique())))
len(list_tfs)

123

### 

In [523]:
# make directory
motif_path = os.path.join(save_dir, 'motif_scan4')
if not os.path.exists(motif_path):
    os.makedirs(motif_path)
print(motif_path)
# input_fasta = '../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_vocab_seq5.fasta'
input_fasta = '../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_vocab_seq4.fasta'
# run_file = 'run_8E1_make_mpra_7_motif_scan.sh'
run_file = 'run_8F1_make_mpra_7_motif_scan_vocab_filt5.sh'
run_file = 'run_8F1_make_mpra_7_motif_scan_vocab_filt4.sh'

with open(run_file, 'w') as f:
# print(len(glob.glob('../data/external/hocomoco_human_trim/*pwm')))
    for pwm_file in sorted(glob.glob('../data/external/hocomoco_human_trim/*pwm')):
        pwm = os.path.basename(pwm_file).split('.pwm')[0]
        tf = tf_id_abbr_to_name_dict[pwm]
        if tf in list_tfs:
            output_file = os.path.join(motif_path, pwm+'_scan.csv')
            cmd = 'moods-dna.py -m {} -s {} -p 0.05 -o {}'.format(pwm_file,input_fasta, output_file) #motif_scan3 uses this, difference from motif_scan4 is the pvalue            
#             cmd = 'moods-dna.py -m {} -s {} -p 0.001 -o {}'.format(pwm_file,input_fasta, output_file) #motif_scan5 
            f.write(cmd+'\n')
# testing
# cmd = 'moods-dna.py -m {} -s {} -p 0.00005 -o {}'.format(pwm_file,input_fasta, output_file)
cmd = 'moods-dna.py -m {} -s {} -p 0.05 -o {}'.format(pwm_file,input_fasta, output_file)
print(cmd)    

### controls

In [622]:
# make directory
motif_path = os.path.join(save_dir, 'motif_scan_control')
if not os.path.exists(motif_path):
    os.makedirs(motif_path)
print(motif_path)
input_fasta = '../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_control_seq.fasta'
run_file = 'run_8F1_make_mpra_7_motif_scan_control.sh'

with open(run_file, 'w') as f:
# print(len(glob.glob('../data/external/hocomoco_human_trim/*pwm')))
    for pwm_file in sorted(glob.glob('../data/external/hocomoco_human_trim/*pwm')):
        pwm = os.path.basename(pwm_file).split('.pwm')[0]
        tf = tf_id_abbr_to_name_dict[pwm]
        if tf in list_tfs:
            output_file = os.path.join(motif_path, pwm+'_scan.csv')
            cmd = 'moods-dna.py -m {} -s {} -p 0.05 -o {}'.format(pwm_file,input_fasta, output_file) #motif_scan3 uses this, difference from motif_scan4 is the pvalue            
#             cmd = 'moods-dna.py -m {} -s {} -p 0.001 -o {}'.format(pwm_file,input_fasta, output_file) #motif_scan5 
            f.write(cmd+'\n')
# testing
# cmd = 'moods-dna.py -m {} -s {} -p 0.00005 -o {}'.format(pwm_file,input_fasta, output_file)
cmd = 'moods-dna.py -m {} -s {} -p 0.05 -o {}'.format(pwm_file,input_fasta, output_file)
print(cmd)    

../data/processed/fig4_modelling/vocab_sum_final_post_scan/motif_scan_control
moods-dna.py -m ../data/external/hocomoco_human_trim/ZSC31_HUMAN.H11MO.0.C.pwm.trim.pwm -s ../data/processed/fig4_modelling/vocab_sum_final_post_scan/mpra_control_seq.fasta -p 0.05 -o ../data/processed/fig4_modelling/vocab_sum_final_post_scan/motif_scan_control/ZN816_HUMAN.H11MO.1.C_scan.csv


in cmd line in this folder run
- `bash run_8E1_make_mpra_7_motif_scan_vocab_filt4.sh`
- `bash run_8E1_make_mpra_7_motif_scan_vocab_filt5.sh`

in sherlock
```
cd /oak/stanford/groups/khavari/users/mguo123/pan_omics/notebooks

sbatch run_8F1_make_mpra_7_motif_scan_vocab_filt4.sbatch



#!/bin/bash
#run_8F1_make_mpra_7_motif_scan4.sbatch
#09/01/2020
#SBATCH -J motif_filt# A single job name for the array
#SBATCH -p rbaltman,khavari # Partition
#SBATCH -n 1 # one core
#SBATCH --mem=64000
#SBATCH --time=48:00:00
#SBATCH -o run_8E1_filt.out # Standard output
#SBATCH -e run_8E1_filt.err # Standard error
#SBATCH --mail-type=FAIL,END # notifications for job done & fail
#SBATCH --mail-user=mguo123@stanford.edu

cd /oak/stanford/groups/khavari/users/mguo123/pan_omics/notebooks
PATH="$PATH:`python -m site --user-base`/bin"

bash run_8F1_make_mpra_7_motif_scan_vocab_filt4.sh

```

#### header of the `*_scan.csv`
   `ARID5B::KLF5+GDSD6+4+A_B,SRBP2_HUMAN.H11MO.0.B.pwm.trim.pwm,73,-,5.35564936116,CCACCCCGC,`
cvs headers (8 cols)
- vocabulary config name (from fasta)
- pwm filename
- position in the sequence (index 0)
- strand: Indicates whether the match is versus the input strand (+) or the reverse complement (-).
- score (bit score)
- sequence substring from what we are scanning (original sequence) (note that if the match is -, then the match is versus the reverse complement of this).
- (NOT USED)Hit sequence with SNPs applied, i.e. shows how the ambiguity codes are resolved to get this score (there can be multiple hits at the same position with different "real" hit sequences).


#### more on the score
    https://www.cs.helsinki.fi/group/pssmfind/doc/python/MOODS.html
      The input PWM matrix transformed to log-odds scores.
      The score for nucleotide N and position i is

         log ( matrix[N][i] + ps * bg[N] ) / C[i]
           - log ( bg[N] ),

      where C[i] is the sum of terms

         matrix[N][i] + ps * bg[N]

      for all nucleotides N.

    so all the matches to the PWM transformed log-odds score are summed to get the match score


## 7C read in motif scan data 

cases for the motif in the 
1. scrambling removes motif 
2. there's a dummy motif somewhere else in the genomic instance (????? How would you deal with these “noisy” cases?
    2A. is the dummy motif is present in all 4 cases, (not changed by scrambling)
    2B. the dummy motif is present in only some cases, so probably changed by scrambling the other site?
3. scrambling reintroduces the motif at a (hopefully) lower strength, and sometimes with a shifted start position  ??? Potential to rescramble (but sometimes motifs have too many repetitive nucleotides so this might not work…
    3A. the scrambling introduces the motif at a lower strength
    3B. the scrambling does not lower the score of the motif
4. we don't detect the motif in any config in the genomic instance at a certain p value threshold (this usually isn’t the case, if we raise the p-value threshold enough; but the motif will be detected in a lot more genomic instances). This is sort of the sensitivity vs specificity tradeoff.


2 parts
1. map the vocab word locations to motifs and match to case above (get the score and the case) - exact match
2. determine vocab words gained lost between the motif configurations

### 7C1: get vocab_instance_df set up with locations and vocab predictions given config

In [395]:
# vocab_oligo_df.columns
vocab_instances20_seqs_sel = pd.read_csv(os.path.join(save_dir, 'vocab_instances20_seq_wscr.csv'),index_col=0)
vocab_instances20_seqs_sel[:5]

,vocab_pair,tot_dist,btn_dist,chr,start_pad,stop_pad,vocab1,vocab1_start_shift,vocab1_stop_shift,vocab2,...,vocab2_stop_shift,tissue,loop_type,vocab_tissue_count,name,check_rs,scr_pass_rs,seq_config,seq,name_config
0,ARID5B::KLF5,17,2,chr14,69253380,69253514,KLF5,54,71,ARID5B,...,85,GDSD6,loop_loop,8,ARID5B::KLF5+GDSD6+8,0,True,A_B_seq,ACTCCCAAAACTTCAAGCTCGAAAACAGGTTGAGGGCAAGTGAGCA...,ARID5B::KLF5+GDSD6+8+A_B
1,ARID5B::KLF5,17,2,chr6,13477381,13477515,ARID5B,51,64,KLF5,...,81,GDSD6,loop_loop,5,ARID5B::KLF5+GDSD6+5,0,True,A_B_seq,CAGGTGGCCTTGATCCAGGCAAAAATTGCGACTGTTTGATATCGCC...,ARID5B::KLF5+GDSD6+5+A_B
2,ARID5B::KLF5,17,2,chr6,13477381,13477515,ARID5B,51,64,KLF5,...,81,GDSD6,pro_pro,6,ARID5B::KLF5+GDSD6+6,0,True,A_B_seq,CAGGTGGCCTTGATCCAGGCAAAACTGGGAGATTGTTATCTTACCC...,ARID5B::KLF5+GDSD6+6+A_B
3,ARID5B::KLF5,18,3,chr17,7145720,7145854,ARID5B,50,63,KLF5,...,81,GDSD6,loop_loop,10,ARID5B::KLF5+GDSD6+10,0,True,A_B_seq,GACGGCGGCGACGCGCGGGCGGATTCAGCGGAGCGATCCACGAATT...,ARID5B::KLF5+GDSD6+10+A_B
4,ARID5B::KLF5,18,3,chr19,14616931,14617065,KLF5,53,70,ARID5B,...,85,GDSD6,loop_loop,7,ARID5B::KLF5+GDSD6+7,0,True,A_B_seq,TCTTCCCCTCCACGCTGCGGGCTAGGATTCGGATGTGGCCTGGATT...,ARID5B::KLF5+GDSD6+7+A_B


In [396]:
def vocab_present_func(row):
    # determine if there should be a vocab present (based on configuration)
    return row['vocab_order'] in row['config'].split('_')
#     vocab_instances_df.vocab_order.apply(lambda x: xisin(vocab_instances_df.config.str.split('_'))

In [397]:
vocab_instances_df = vocab_instances20_seqs_sel[['name_config','chr','start_pad','stop_pad','vocab1','vocab1_start_shift','vocab1_stop_shift']]
vocab_instances_df.columns = ['name_config','chr','start_pad','stop_pad','vocab','vocab_start','vocab_stop']
vocab_instances_df['vocab_order'] = 'A'

vocab_instances_df2 = vocab_instances20_seqs_sel[['name_config','chr','start_pad','stop_pad','vocab2','vocab2_start_shift','vocab2_stop_shift']]
vocab_instances_df2.columns = ['name_config','chr','start_pad','stop_pad','vocab','vocab_start','vocab_stop']
vocab_instances_df2['vocab_order'] = 'B'
vocab_instances_df = pd.concat([vocab_instances_df,vocab_instances_df2],axis=0,sort=True)
vocab_instances_df['instance'] = vocab_instances_df.name_config.apply(lambda x: '+'.join(x.split('+')[:-1]))
vocab_instances_df['config'] = vocab_instances_df.name_config.str.split('+',expand=True).iloc[:,-1]
vocab_instances_df['vocab_present_pred'] = vocab_instances_df.apply(vocab_present_func,axis=1)
vocab_instances_df['vocab_loc_start'] = vocab_instances_df.vocab_start# - vocab_instances_df.start_pad
vocab_instances_df['vocab_loc_stop'] = vocab_instances_df.vocab_stop #- vocab_instances_df.start_pad
vocab_instances_df.sort_values('name_config',inplace=True)
vocab_instances_df

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,chr,name_config,start_pad,stop_pad,vocab,vocab_order,vocab_start,vocab_stop,instance,config,vocab_present_pred,vocab_loc_start,vocab_loc_stop
7,chr18,ARID5B::KLF5+GDSD6+1+A_B,61243074,61243208,KLF5,B,62,83,ARID5B::KLF5+GDSD6+1,A_B,True,62,83
7,chr18,ARID5B::KLF5+GDSD6+1+A_B,61243074,61243208,ARID5B,A,49,62,ARID5B::KLF5+GDSD6+1,A_B,True,49,62
2397,chr18,ARID5B::KLF5+GDSD6+1+A_scrB,61243074,61243208,ARID5B,A,49,62,ARID5B::KLF5+GDSD6+1,A_scrB,True,49,62
2397,chr18,ARID5B::KLF5+GDSD6+1+A_scrB,61243074,61243208,KLF5,B,62,83,ARID5B::KLF5+GDSD6+1,A_scrB,False,62,83
4787,chr18,ARID5B::KLF5+GDSD6+1+scrA_B,61243074,61243208,ARID5B,A,49,62,ARID5B::KLF5+GDSD6+1,scrA_B,False,49,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,chr7,ZFX::E2F6+SCC+9+A_scrB,44153033,44153167,E2F6,A,47,68,ZFX::E2F6+SCC+9,A_scrB,True,47,68
7168,chr7,ZFX::E2F6+SCC+9+scrA_B,44153033,44153167,ZFX,B,68,85,ZFX::E2F6+SCC+9,scrA_B,True,68,85
7168,chr7,ZFX::E2F6+SCC+9+scrA_B,44153033,44153167,E2F6,A,47,68,ZFX::E2F6+SCC+9,scrA_B,False,47,68
9558,chr7,ZFX::E2F6+SCC+9+scrA_scrB,44153033,44153167,E2F6,A,47,68,ZFX::E2F6+SCC+9,scrA_scrB,False,47,68


In [398]:
vocab_instances_df.instance.unique().shape

(2390,)

### 7C2: add in motif_scan information per case

In [585]:
# helper functions
def label_config(row):
    if row['vocab_present_actual']: 
        if row['vocab_present_pred']:
            return 'TP' # good, Case 1
        else:
            return 'FP' # noisy, Case 3
    else:
        if row['vocab_present_pred']:
            return 'FN' # not detected (Case 4)
        else:
            return 'TN' # good, Case 1
def get_case_instance(row):
    row_counts = row.value_counts()
    # case 1: all the configs that should have the vocab have the vocab and those that don't do not
    if (row_counts.get('TP',0) +row_counts.get('TN',0) )==4:
        return 'case 1'
    # case 4: the pvalue threshold was too stringent
    elif (row_counts.get('FN',0) +row_counts.get('TN',0) )==4:
        return 'case 4'
    # case 3: too noisy
    elif (row_counts.get('TP',0) +row_counts.get('FP',0) )==4:
        return 'case 3'
    else:
        return 'other'

def analyze_motif_scan(motif_scan_file, tf_id_abbr_to_name_dict=tf_id_abbr_to_name_dict):
    # read in motif scan file
    motif_scan_results = pd.read_csv(motif_scan_file,names=['name_config','pwm_file','pos','strand','score','seq','seq_actual'])
    motif = tf_id_abbr_to_name_dict[os.path.basename(motif_scan_file).split('_scan')[0]]
    motif_scan_results['motif'] = motif_scan_results.pwm_file.str.split('.pwm',expand=True).iloc[:,0].map(tf_id_abbr_to_name_dict)

    # merge in genomic location information
    vocab_motif_scan = vocab_instances_df[vocab_instances_df.vocab==motif].merge( motif_scan_results[['name_config','pos','score','motif']], how='left',left_on=['name_config','vocab'], right_on=['name_config','motif'])
    vocab_motif_scan.pos.fillna(-1.0, inplace=True)
    vocab_motif_scan.score.fillna(0, inplace=True)
    vocab_motif_scan.motif.fillna('', inplace=True)
    vocab_motif_scan['vocab_present_actual'] = abs(vocab_motif_scan.vocab_loc_start - vocab_motif_scan.pos ) <=3
    vocab_motif_scan['off_target']  =  ~( ( vocab_motif_scan.vocab_present_actual) | (vocab_motif_scan.pos<0))

    #### case 2: off-target effects
    vocab_motif_scan_off_target = vocab_motif_scan[vocab_motif_scan.off_target]
    vocab_motif_scan_off_target.sort_values('score',ascending=False,inplace=True)
    vocab_motif_scan_off_target.drop_duplicates('name_config',inplace=True)
    vocab_motif_scan_off_target.sort_values('name_config',inplace=True)
    vocab_motif_scan_off_target['pos_close'] = vocab_motif_scan_off_target.pos - (vocab_motif_scan_off_target.pos%2)
    off_target_counts_df = pd.DataFrame(vocab_motif_scan_off_target.groupby('instance').pos_close.value_counts())
    off_target_counts_df.columns = ['count_pos']
    off_target_counts_df['motif'] = motif
    off_target_counts_df['case'] = off_target_counts_df.count_pos.apply(lambda x: 'case 2A' if x==4 else 'case 2B')
    off_target_counts_df.reset_index(inplace=True)

    ##### Case 1,3,4
    # - case 1: good
    # - case 3: FP (too noisy, scrambling might not be good), case 4: FN (pval, threshold too low )

    # only include the highest scoring 
    vocab_motif_scan_on_target = vocab_motif_scan[~vocab_motif_scan.off_target]
    vocab_motif_scan_on_target.sort_values('score',ascending=False,inplace=True)
    vocab_motif_scan_on_target.drop_duplicates('name_config',inplace=True)
    vocab_motif_scan_on_target.sort_values('name_config',inplace=True)

    # fill in all possible instances in the library
    vocab_motif_scan_on_target = vocab_instances_df[vocab_instances_df.vocab==motif].merge(vocab_motif_scan_on_target,how='left',on=list(vocab_instances_df.columns.values))
    vocab_motif_scan_on_target.pos.fillna(-1.0, inplace=True)
    vocab_motif_scan_on_target.score.fillna(0, inplace=True)
    vocab_motif_scan_on_target.motif.fillna('', inplace=True)
    vocab_motif_scan_on_target.vocab_present_actual.fillna(False, inplace=True)
    vocab_motif_scan_on_target.off_target.fillna(False, inplace=True)
    vocab_motif_scan_on_target['eval'] = vocab_motif_scan_on_target.apply(label_config, axis=1)

    # create data frame thats has rows (instances) and columns are the eval (TP,FP, etc) and score per config
    vocab_motif_scan_on_target_config = vocab_motif_scan_on_target.pivot(index='instance', columns='config', values=['eval','score'])
    vocab_motif_scan_on_target_config.columns = ['_'.join(list(x)) for x in vocab_motif_scan_on_target_config.columns.values]
    vocab_motif_scan_on_target_config['motif'] = motif
    vocab_motif_scan_on_target_config['case'] = vocab_motif_scan_on_target_config.apply(get_case_instance,axis=1)

    # dict of case_type: # instances that fall in type
    on_target_config_cases = vocab_motif_scan_on_target_config.case.value_counts()
    
    return motif, off_target_counts_df, vocab_motif_scan_on_target_config, on_target_config_cases

def analyze_motif_scan_wrap(motif_scan_files, save_prefix=None):
    print('# motif files: ', len(motif_scan_files))
    # overall dataframe setup
    off_target_counts_df_all = pd.DataFrame()
    on_target_cases_df_all = pd.DataFrame()
    on_target_config_cases_dict = {}
    
    #get scan results from file
    for idx,motif_scan_file in enumerate(motif_scan_files):
        ## DEBUG
    #     print(idx,motif_scan_file)
        motif, off_target_counts_df, vocab_motif_scan_on_target_config, on_target_config_cases = analyze_motif_scan(motif_scan_file)
        # ADD TO RUNNING LIST (ACROSS MOTIFS)
        off_target_counts_df_all = pd.concat([off_target_counts_df_all, off_target_counts_df],axis=0,sort=True)
        on_target_cases_df_all = pd.concat([on_target_cases_df_all, vocab_motif_scan_on_target_config],axis=0,sort=True)
        on_target_config_cases_dict[motif] = on_target_config_cases

    # convert dict to dataframe (row: motif, columns case type)
    on_target_config_cases_df = pd.DataFrame.from_dict(on_target_config_cases_dict,orient='index').fillna(0)

    ## TODO: add in save functionality
    if save_prefix is not None:
        off_target_counts_df_all.to_csv(save_prefix+'off_target_counts_df_all.csv')
        on_target_cases_df_all.to_csv(save_prefix+'motif_scan_'+'on_target_cases_df_all.csv')
        on_target_config_cases_df.to_csv(save_prefix+ 'motif_scan_'+'on_target_config_cases_df.csv')
        print('saved', save_prefix+'*.csv')
    return off_target_counts_df_all, on_target_cases_df_all, on_target_config_cases_df

def refine_case3(row):
    if row['case']!='case 3':
#         print(row['case'])
        return row['case']
    else:
        # only remodify case 3:
        if row['score_A_B']> min(row['score_A_scrB'],row['score_scrA_B'] ):
    #         print('here')
            return 'case 3A' # this is the good case where scrambling reduces the score of the motif
        else:
            return 'case 3B'
        # case 3, where the TP are have higher score than where the true negatives are

def get_offtarget_num(off_target_counts_df= off_target_counts_df_all_5, vocab_instances_df=vocab_instances20_seqs_sel):
    off_target_res_case_2A = off_target_counts_df[off_target_counts_df.case=='case 2A']
    print(off_target_res_case_2A.shape)
    off_target_res_case_2A['off_target_pos_str'] = off_target_res_case_2A.pos_close.map(int).map(str)
    off_target_res_case_2A = off_target_res_case_2A.groupby(['instance','motif']).agg({'off_target_pos_str':'|'.join}).reset_index()
    off_target_res_case_2A.columns = ['off_target_name','off_target_motif','off_target_pos_str']
    print(off_target_res_case_2A.shape) # groupbed by instance
    vocab_instances20_seqs_sel_offtarget = vocab_instances_df.merge(off_target_res_case_2A, how='left',left_on='name',right_on='off_target_name')
    vocab_instances20_seqs_sel_offtarget.off_target_name.fillna('',inplace=True)
    vocab_instances20_seqs_sel_offtarget.off_target_motif.fillna('',inplace=True)
    vocab_instances20_seqs_sel_offtarget.off_target_pos_str.fillna('',inplace=True)
    vocab_instances20_seqs_sel_offtarget = pd.concat([vocab_instances20_seqs_sel_offtarget,vocab_instances20_seqs_sel_offtarget.apply(get_vocab_loc,axis=1)],axis=1)
    
#     return vocab_instances20_seqs_sel_offtarget
    vocab_instances20_seqs_sel_offtarget = vocab_instances20_seqs_sel_offtarget.groupby(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
           'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
           'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type', 'name', 'check_rs', 'scr_pass_rs', 
           'vocab_tissue_count', 'seq_config', 'seq','name_config', 'off_target_name','vocab1_start_shift', 'vocab1_stop_shift',
           'vocab2_start_shift', 'vocab2_stop_shift']).agg({'off_target_motif':'::'.join,'off_target_pos_str':'::'.join}).reset_index()

    vocab_instances20_seqs_sel_offtarget = vocab_instances20_seqs_sel_offtarget.groupby(['vocab_pair', 'tot_dist', 'btn_dist', 'chr', 'start', 'stop',
           'start_pad', 'stop_pad', 'vocab1', 'vocab1_start', 'vocab1_stop',
           'vocab2', 'vocab2_start', 'vocab2_stop', 'tissue', 'loop_type', 'name',
           'check_rs', 'scr_pass_rs', 'vocab_tissue_count',  'vocab1_start_shift',
           'vocab1_stop_shift', 'vocab2_start_shift', 'vocab2_stop_shift','off_target_name',
           'off_target_motif', 'off_target_pos_str']).agg({'seq_config':list,'seq':list, 'name_config':list}).reset_index()
    # 


    off_target_num_df = pd.DataFrame(vocab_instances20_seqs_sel_offtarget.set_index('name').off_target_pos_str.apply(off_target_pos_str_find_num)).reset_index()
    off_target_num_df['instance'] = off_target_num_df.name.apply(lambda x: '+'.join(x.split('+')[:-1]))
    off_target_num_df_counts = off_target_num_df[off_target_num_df.off_target_pos_str==0].instance.value_counts().value_counts().sort_index()
    print(off_target_num_df_counts/off_target_num_df_counts.sum())
    return off_target_num_df[off_target_num_df.off_target_pos_str==0].instance.value_counts().describe()



def off_target_pos_str_find_num(pos_str):
    pos_arr_per_vocab_word = pos_str.split('::')
    if len(pos_arr_per_vocab_word)>0:
        arr = []
        for pos in pos_arr_per_vocab_word:
            for x in pos.split(':'):
                try:
                    arr.append(int(x))
                except:
                    continue
        return len(set(arr))
    return 0

def on_target_new_cases(row):
    motif_arr = row['motif'].split('|')
    case_mod_arr = row['case_mod'].split('|')
    num_motifs = len(set(motif_arr))
    if num_motifs>2:
        return '|'.join(sorted(set(motif_arr))), 'more than 2 motifs'
    elif num_motifs<2:
        return '|'.join(sorted(set(motif_arr))), 'less than 2 motifs'    
    else:
        motif_to_case = {}
        for motif, case in zip(motif_arr, case_mod_arr):
            if motif in motif_to_case:
                if (motif_to_case[motif]!='case 3A') & (motif_to_case[motif]!='case 1'):
                    motif_to_case[motif]=case
            else:
                motif_to_case[motif]=case
        motif_new = '|'.join(list(motif_to_case.keys()))
        case_new = '|'.join(list(motif_to_case.values()))
        return motif_new, case_new
    
def on_target_new_case_good(case_new):
    if '|' not in case_new:
        return False # not two motifs
    else:
        case_status = True
        for case in case_new.split('|'):
            if  (case!='case 3A') & (case!='case 1') :
                case_status = False
        return case_status
    
def get_on_target_good_instance_num(on_target_cases_df):
    on_target_counts = on_target_cases_df.reset_index()[['instance','motif','case_mod']].drop_duplicates().groupby('instance').agg({
    'motif':'|'.join, 'case_mod':'|'.join
    }).reset_index()
    on_target_counts['name'] = on_target_counts.instance.apply(lambda x: '+'.join(x.split('+')[:-1]))
    on_target_counts = pd.concat([on_target_counts, on_target_counts.apply(on_target_new_cases,axis=1).apply(lambda s: pd.Series({'motif_new':s[0], 'case_new':s[1]}))],axis=1)
    on_target_counts['case_good_bool'] = on_target_counts.case_new.apply(on_target_new_case_good)
    return on_target_counts[on_target_counts.case_good_bool==True].name.value_counts().describe()

#### version 1: homotypic scrambling with pval <0.05

In [684]:
%%time
# takes ~ 15s

motif_scan_files4 = sorted(glob.glob(os.path.join(save_dir, 'motif_scan4','*csv')))
off_target_counts_df_all_4, on_target_cases_df_all_4, on_target_config_cases_df_4 = analyze_motif_scan_wrap(motif_scan_files4, save_prefix=os.path.join(save_dir, 'motif_scan4_'))
on_target_config_cases_df_4.sum(axis=0)

# motif files:  158


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipyk

saved ../data/processed/fig4_modelling/vocab_sum_final_post_scan/motif_scan4_*.csv
CPU times: user 1min 58s, sys: 3.53 s, total: 2min 2s
Wall time: 2min 2s


case 3    2153.0
case 1     986.0
other      898.0
case 4     743.0
dtype: float64

In [685]:
on_target_cases_df_all_4.case.value_counts()

case 3    2852
case 1    1324
other     1043
case 4     861
Name: case, dtype: int64

In [686]:
on_target_cases_df_all_4['case_mod'] = on_target_cases_df_all_4.apply(refine_case3, axis=1)
on_target_cases_df_all_4.case_mod.value_counts()

case 3A    1905
case 1     1324
other      1043
case 3B     947
case 4      861
Name: case_mod, dtype: int64

In [687]:
on_target_cases_df_all_4[on_target_cases_df_all_4.case=='other'][['eval_A_B',
                                                                  'eval_A_scrB',	
                                                                  'eval_scrA_B',	
                                                                  'eval_scrA_scrB']].apply('|'.join,axis=1).value_counts()

FN|FN|FP|FP    467
FN|FP|FN|FP    359
TP|FP|TP|TN     55
TP|TN|TP|FP     52
TP|FP|FN|FP     17
FN|TN|TP|TN     15
FN|TN|FN|FP     14
FN|FP|FN|TN     13
TP|TN|FN|TN      9
FN|FP|TP|FP      8
TP|TP|FP|TN      6
FN|TN|TP|FP      6
TP|TP|TN|FP      5
TP|FN|FP|FP      4
FN|FN|TN|FP      3
TP|FP|FN|TN      3
FN|TP|FP|FP      2
FN|TP|TN|TN      2
TP|FN|FP|TN      1
FN|FN|FP|TN      1
FN|TP|FP|TN      1
dtype: int64

In [688]:
get_on_target_good_instance_num(on_target_cases_df_all_4)


count    208.000000
mean       3.495192
std        2.206704
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max       10.000000
Name: name, dtype: float64

In [689]:
on_target_config_cases_df_4.sum(axis=0).sum() #double the number of instances b/c of the two vocab words per instance

4780.0

In [690]:
off_target_counts_df_all_4.reset_index()[['instance','motif']].drop_duplicates().shape

(4777, 2)

In [691]:
off_target_counts_df_all_4[off_target_counts_df_all_4.case=='case 2A'][['instance','motif']].drop_duplicates().shape

(813, 2)

In [692]:
print(off_target_counts_df_all_4[['instance','motif','case']].drop_duplicates().shape)
off_target_counts_df_all_4[['instance','motif','case']].drop_duplicates()['case'].value_counts()

(5141, 3)


case 2B    4328
case 2A     813
Name: case, dtype: int64

In [693]:
get_offtarget_num(off_target_counts_df_all_4)

(888, 5)
(813, 3)


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1     0.004184
2     0.008368
3     0.033473
4     0.050209
5     0.104603
6     0.150628
7     0.163180
8     0.209205
9     0.175732
10    0.100418
Name: instance, dtype: float64


count    239.000000
mean       7.150628
std        1.930056
min        1.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       10.000000
Name: instance, dtype: float64

#### version 2: homotypic scrambling with pval <0.001

In [623]:
%%time
# takes ~ 15s
motif_scan_files5 = sorted(glob.glob(os.path.join(save_dir, 'motif_scan5','*csv')))
off_target_counts_df_all_5, on_target_cases_df_all_5, on_target_config_cases_df_5 = analyze_motif_scan_wrap(motif_scan_files5, save_prefix=os.path.join(save_dir, 'motif_scan5_'))
on_target_config_cases_df_5.sum(axis=0)

# motif files:  158


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipyk

saved ../data/processed/fig4_modelling/vocab_sum_final_post_scan/motif_scan5_*.csv
CPU times: user 14.1 s, sys: 147 ms, total: 14.3 s
Wall time: 14.3 s


case 4    3559.0
case 1     817.0
other      277.0
case 3     127.0
dtype: float64

In [624]:
on_target_cases_df_all_5.case.value_counts()

case 4    4413
case 1    1152
other      333
case 3     182
Name: case, dtype: int64

In [625]:
on_target_cases_df_all_5['case_mod'] = on_target_cases_df_all_5.apply(refine_case3, axis=1)
on_target_cases_df_all_5.case_mod.value_counts()

case 4     4413
case 1     1152
other       333
case 3A     116
case 3B      66
Name: case_mod, dtype: int64

In [683]:
get_on_target_good_instance_num(on_target_cases_df_all_5)

count    74.000000
mean      1.905405
std       1.206931
min       1.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       7.000000
Name: name, dtype: float64

In [626]:
on_target_config_cases_df_5.sum(axis=0).sum() #double the number of instances b/c of the two vocab words per instance

4780.0

In [627]:
off_target_counts_df_all_5.reset_index()[['instance','motif']].drop_duplicates().shape

(3871, 2)

In [628]:
print(off_target_counts_df_all_5[['instance','motif','case']].drop_duplicates().shape)
off_target_counts_df_all_5[['instance','motif','case']].drop_duplicates()['case'].value_counts()

(3931, 3)


case 2B    3687
case 2A     244
Name: case, dtype: int64

In [654]:
get_offtarget_num(off_target_counts_df_all_5)

(258, 5)
(244, 3)


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4     0.004184
6     0.008368
7     0.054393
8     0.179916
9     0.347280
10    0.405858
Name: instance, dtype: float64


count    239.000000
mean       9.071130
std        0.991118
min        4.000000
25%        9.000000
50%        9.000000
75%       10.000000
max       10.000000
Name: instance, dtype: float64